In [7]:
# =============================================
# TP5 - Pipeline de NLP (Tarefas 1→7 em bloco único)
# ——— Envio inicial: apenas TAREFA 1 (TF-IDF) ———
# Observação: este bloco já está estruturado para
# receber as Tarefas 2–7 sem duplicações.
# =============================================

from __future__ import annotations

# =========[SEÇÃO: IMPORTS ATUAIS — usados na Tarefa 1]=========
import os
import re
import json
import pickle
import string
from dataclasses import dataclass
from typing import Tuple, List, Dict, Any
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel


# =============================================
# [SEÇÃO A] Configurações gerais

SEMENTE_GLOBAL: int = 42
np.random.seed(SEMENTE_GLOBAL)

@dataclass(frozen=True)
class ConfigProjeto:
    caminho_csv: str = "IMDB Dataset.csv"     # arquivo no mesmo diretório do .ipynb
    coluna_texto: str = "review"
    coluna_alvo: str = "sentiment"
    salvar_dir: str = "./artefatos"           # pasta local para artefatos
    # Hiperparâmetros do TF-IDF (pensados para classificação e t-SNE;
    # LDA usará BoW específico na Tarefa 2)
    max_features_tfidf: int = 50000
    max_df: float = 0.9
    min_df: int | float = 5
    ngram_range: Tuple[int, int] = (1, 2)
    norm: str = "l2"
    stop_words: str | None = "english"
    lowercase: bool = True

CFG = ConfigProjeto()

# =============================================
# [SEÇÃO B] Pré-processamento textual
# =============================================

_regex_html = re.compile(r"<.*?>")
_regex_url  = re.compile(r"http\S+|www\.\S+")
_regex_num  = re.compile(r"\d+")
_tbl_traducao_punct = str.maketrans("", "", string.punctuation)

def limpar_texto(texto: str) -> str:
    """
    Limpeza leve para IMDB:
    - remove tags HTML, URLs, números, pontuação
    - normaliza espaços
    Evita stem/lemma nesta etapa para não atrapalhar análises futuras (ex.: LDA).
    """
    if not isinstance(texto, str):
        return ""
    t = texto
    t = _regex_html.sub(" ", t)
    t = _regex_url.sub(" ", t)
    t = _regex_num.sub(" ", t)
    t = t.translate(_tbl_traducao_punct)
    t = re.sub(r"\s+", " ", t, flags=re.MULTILINE).strip()
    return t

# =============================================
# [SEÇÃO C] Carregamento da base
# =============================================

def carregar_base(caminho: str, coluna_texto: str, coluna_alvo: str) -> pd.DataFrame:
    if not os.path.exists(caminho):
        raise FileNotFoundError(f"Base não encontrada em: {caminho}")
    df = pd.read_csv(caminho)
    colunas_necessarias = {coluna_texto, coluna_alvo}
    ausentes = colunas_necessarias - set(df.columns)
    if ausentes:
        raise ValueError(f"Colunas ausentes no CSV: {ausentes}")
    return df[[coluna_texto, coluna_alvo]].copy()

# =============================================
# [SEÇÃO D] Criação das features — TF-IDF (Tarefa 1)
# =============================================

def criar_tfidf(textos: List[str], cfg: ConfigProjeto) -> tuple[TfidfVectorizer, Any]:
    """
    Ajusta TfidfVectorizer sobre o corpus.
    Retorna (vetorizador, matriz_esparsa_TF-IDF).
    """
    vetorizador = TfidfVectorizer(
        lowercase=cfg.lowercase,
        stop_words=cfg.stop_words,
        max_df=cfg.max_df,
        min_df=cfg.min_df,
        ngram_range=cfg.ngram_range,
        max_features=cfg.max_features_tfidf,
        norm=cfg.norm
    )
    matriz = vetorizador.fit_transform(textos)
    return vetorizador, matriz

def salvar_artefatos_tfidf(vetorizador: TfidfVectorizer, X_esparsa, cfg: ConfigProjeto) -> Dict[str, str]:
    """
    Salva:
      - Vetorizador (PKL)
      - Vocabulário (CSV)
      - Metadados da matriz (JSON: shape, nnz, densidade)
    Evita salvar a matriz completa (IO pesado).
    """
    os.makedirs(cfg.salvar_dir, exist_ok=True)
    caminhos: Dict[str, str] = {}

    # Vetorizador
    caminho_vec = os.path.join(cfg.salvar_dir, "vetorizador_tfidf.pkl")
    with open(caminho_vec, "wb") as f:
        pickle.dump(vetorizador, f)
    caminhos["vetorizador"] = caminho_vec

    # Vocabulário (ordenado por índice interno)
    vocab = pd.DataFrame({
        "termo": list(vetorizador.vocabulary_.keys()),
        "indice": list(vetorizador.vocabulary_.values())
    }).sort_values("indice").reset_index(drop=True)
    caminho_vocab = os.path.join(cfg.salvar_dir, "vocabulario_tfidf.csv")
    vocab.to_csv(caminho_vocab, index=False, encoding="utf-8")
    caminhos["vocabulario"] = caminho_vocab

    # Metadados da matriz
    meta = {
        "shape": list(X_esparsa.shape),
        "nnz": int(X_esparsa.nnz),
        "densidade": float(X_esparsa.nnz / (X_esparsa.shape[0] * X_esparsa.shape[1]))
    }
    caminho_meta = os.path.join(cfg.salvar_dir, "tfidf_meta.json")
    with open(caminho_meta, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    caminhos["metadados"] = caminho_meta

    return caminhos

# =============================================
# [SEÇÃO E] Execução — Tarefa 1
# =============================================

# 1) Carrega base
df_raw = carregar_base(CFG.caminho_csv, CFG.coluna_texto, CFG.coluna_alvo)

# 2) Limpa textos
df = df_raw.copy()
df["texto_limpo"] = df[CFG.coluna_texto].apply(limpar_texto)

# 3) Cria TF-IDF (fit no corpus completo)
vetorizador_tfidf, X_tfidf = criar_tfidf(df["texto_limpo"].tolist(), CFG)

# 4) (Opcional) Codifica rótulos para uso futuro em classificadores
y_alvo = df[CFG.coluna_alvo].astype("category").cat.codes  # negativo/positivo -> 0/1

# 5) Salva artefatos para reuso nas Tarefas 2–7
caminhos = salvar_artefatos_tfidf(vetorizador_tfidf, X_tfidf, CFG)

# 6) Relatórios rápidos
num_docs, num_feats = X_tfidf.shape
densidade = X_tfidf.nnz / (num_docs * num_feats)

print("=== [Tarefa 1] TF-IDF concluída ===")
print(f"Documentos: {num_docs:,}")
print(f"Features (vocabulário): {num_feats:,}")
print(f"Densidade média da matriz: {densidade:.6f}")
print(f"Classes codificadas encontradas: {sorted(set(y_alvo.tolist()))} -> 0=negativo, 1=positivo")
print("\nArquivos salvos:")
for k, v in caminhos.items():
    print(f"- {k}: {v}")

# - Tarefa 2 (LDA): usaremos CountVectorizer/BoW específico para tópicos, reaproveitando df['texto_limpo'].

# =============================================
# [SEÇÃO F] Tarefa 2 — Modelagem de Tópicos com LDA
# (Sequencial ao código da Tarefa 1, MESMO BLOCO)
# - Usa df['texto_limpo'] já criado.
# - Cria representação BoW (CountVectorizer) própria para LDA.
# - Seleciona o nº de tópicos via métrica de coerência (c_v).
# - Salva artefatos para as próximas tarefas.
# =============================================


# ---------------------------------------------
# Configurações específicas da Tarefa 2
# ---------------------------------------------
class ConfigLDA:
    # candidatos de nº de tópicos; ajuste conforme necessidade/tempo
    candidatos_k = [10, 15, 20]
    max_iter = 10
    learning_method = "batch"   # determinístico; "online" é mais rápido
    learning_decay = 0.7
    learning_offset = 10.0
    max_features_bow = 50000
    min_df = 5
    max_df = 0.9
    n_top_palavras = 12         # nº de termos por tópico no print/artefatos

CFG_LDA = ConfigLDA()

# ---------------------------------------------
# Tokenização simples (compatível com a limpeza já feita)
# ---------------------------------------------
def tokenize(texto: str) -> list[str]:
    # df['texto_limpo'] já removeu pontuação, números, links e normalizou espaços
    return texto.lower().split()

# ---------------------------------------------
# Cria representação BoW específica para LDA
# ---------------------------------------------
vectorizador_bow = CountVectorizer(
    lowercase=True,
    stop_words="english",                 # corpus é EN
    max_df=CFG_LDA.max_df,
    min_df=CFG_LDA.min_df,
    max_features=CFG_LDA.max_features_bow
)
X_bow = vectorizador_bow.fit_transform(df["texto_limpo"].tolist())


tokens_corpus = [tokenize(t) for t in df["texto_limpo"].tolist()]
dicionario = Dictionary(tokens_corpus)
# Para coerência c_v com modelos externos (sklearn), passamos textos e dicionário; corpus não é obrigatório.

# ---------------------------------------------
# Função utilitária: extrair tópicos (palavras) de um LDA sklearn
# ---------------------------------------------
def extrair_top_palavras(modelo_lda: LatentDirichletAllocation, vectorizer: CountVectorizer, n_top: int) -> list[list[str]]:
    termos = vectorizer.get_feature_names_out()
    topicos = []
    for k in range(modelo_lda.components_.shape[0]):
        pesos = modelo_lda.components_[k]
        idx = pesos.argsort()[-n_top:][::-1]
        topicos.append([termos[i] for i in idx])
    return topicos

# ---------------------------------------------
# Seleção de K por coerência (c_v)
# ---------------------------------------------
resultados_coerencia = []
melhor_k = None
melhor_score = -np.inf
melhor_modelo = None

for k in CFG_LDA.candidatos_k:
    lda = LatentDirichletAllocation(
        n_components=k,
        max_iter=CFG_LDA.max_iter,
        learning_method=CFG_LDA.learning_method,
        learning_decay=CFG_LDA.learning_decay,
        learning_offset=CFG_LDA.learning_offset,
        random_state=SEMENTE_GLOBAL,
        evaluate_every=-1,
        verbose=0
    )
    lda.fit(X_bow)

    # Obtém top palavras por tópico
    top_palavras = extrair_top_palavras(lda, vectorizador_bow, CFG_LDA.n_top_palavras)

    # Coerência c_v (usa ngrams, indireta; robusta para tópicos interpretáveis)
    cm = CoherenceModel(
        topics=top_palavras,
        texts=tokens_corpus,
        dictionary=dicionario,
        coherence="c_v"
    )
    score_cv = cm.get_coherence()
    resultados_coerencia.append({"k": k, "coerencia_c_v": float(score_cv)})

    if score_cv > melhor_score:
        melhor_score = score_cv
        melhor_k = k
        melhor_modelo = lda

# ---------------------------------------------
# Relatório e salvamento de artefatos
# ---------------------------------------------
print("=== [Tarefa 2] LDA — Seleção de Tópicos por Coerência (c_v) ===")
for r in resultados_coerencia:
    print(f"K={r['k']:>2}  |  Coerência (c_v) = {r['coerencia_c_v']:.4f}")
print(f"\nMelhor K selecionado: {melhor_k}  (c_v={melhor_score:.4f})")

topicos_melhor = extrair_top_palavras(melhor_modelo, vectorizador_bow, CFG_LDA.n_top_palavras)
print("\nTop palavras por tópico (melhor K):")
for i, palavras in enumerate(topicos_melhor, 1):
    print(f" - Tópico {i:02d}: {', '.join(palavras)}")

os.makedirs(CFG.salvar_dir, exist_ok=True)

caminho_vec_bow = os.path.join(CFG.salvar_dir, "vectorizador_bow.pkl")
with open(caminho_vec_bow, "wb") as f:
    pickle.dump(vectorizador_bow, f)

caminho_lda = os.path.join(CFG.salvar_dir, "lda_model.pkl")
with open(caminho_lda, "wb") as f:
    pickle.dump(melhor_modelo, f)

# Tópicos e coerências
caminho_topicos = os.path.join(CFG.salvar_dir, "lda_topicos.csv")
pd.DataFrame(
    {"topico": np.arange(1, len(topicos_melhor) + 1),
     "palavras": [", ".join(p) for p in topicos_melhor]}
).to_csv(caminho_topicos, index=False, encoding="utf-8")

caminho_coerencia = os.path.join(CFG.salvar_dir, "lda_coerencia.json")
with open(caminho_coerencia, "w", encoding="utf-8") as f:
    json.dump(
        {"resultados": resultados_coerencia, "melhor_k": melhor_k, "coerencia_c_v": melhor_score},
        f, ensure_ascii=False, indent=2
    )

print("\nArquivos salvos (Tarefa 2):")
print(f"- vectorizador_bow.pkl: {caminho_vec_bow}")
print(f"- lda_model.pkl:        {caminho_lda}")
print(f"- lda_topicos.csv:      {caminho_topicos}")
print(f"- lda_coerencia.json:   {caminho_coerencia}")



# - Tarefa 3–4 (Classificação/Avaliação): X_tfidf e y_alvo já estão prontos.
# - Tarefa 5 (t-SNE): projete amostras de X_tfidf para 2D (cuidado com custo).
# - Tarefa 6 (LIME/SHAP/force-plot): usar o melhor classificador ajustado sobre X_tfidf.
# - Tarefa 7 (Conclusões): sumarize achados de tópicos e desempenho do classificador.


ModuleNotFoundError: No module named 'gensim'